In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

### load comp with permno data

In [24]:
comp = pd.read_csv('data/fundamental_final_1.2.zip',
                  parse_dates=['datadate', 'fysm'], infer_datetime_format=True)
comp.datadate = pd.to_datetime(comp.datadate)
comp.fysm = pd.to_datetime(comp.fysm)

In [21]:
comp.head()

,gvkey,datadate,fyear,fystr,fysm,permno
0,1000,1973-12-31,1973.0,A-Dec,1973-01-01,25881
1,1000,1974-12-31,1974.0,A-Dec,1974-01-01,25881
2,1000,1975-12-31,1975.0,A-Dec,1975-01-01,25881
3,1000,1976-12-31,1976.0,A-Dec,1976-01-01,25881
4,1000,1977-12-31,1977.0,A-Dec,1977-01-01,25881


In [18]:
crsp = pd.read_csv('data/return_ensembled.zip',
                   parse_dates=['date'], infer_datetime_format=True)
crsp.date = pd.to_datetime(crsp.date)
crsp = crsp.set_index('date')

In [29]:
crsp.head()

,permno,mv,mv_adj,ret_p1,retx_p1,dret_p1
date,,,,,,
1973-01-31,63845,106005.375,526734.998484,0.724138,0.724138,0.945299
1973-01-31,15580,4425.000,21987.586651,1.043478,1.043478,0.999527
1973-01-31,28820,6768.750,33633.554157,1.187500,1.187500,0.988575
1973-01-31,11253,3038.000,15095.658361,0.942308,0.942308,0.999527
1973-01-31,61903,766.500,3808.697213,0.954545,0.954545,1.015606


In [20]:
def fiscal_resample(df, fys, per):
    
#     ret_p1 = df.ret_p1.cumprod().iloc[-1]
#     retx_p1 = df.retx_p1.cumprod().iloc[-1]
#     dret_p1 = df.dret_p1.cumprod().iloc[-1]
    
    ret_p1 = df.ret_p1.prod().iloc[-1]
    retx_p1 = df.retx_p1.prod().iloc[-1]
    dret_p1 = df.dret_p1.prod().iloc[-1]
    
    mv = df.mv.resample(fys).last().iloc[-1]
    mv_adj = df.mv_adj.resample(fys).last().iloc[-1]
    
    res = pd.DataFrame({'ret_p1':ret_p1, 'retx_p1':retx_p1, 'dret_p1':dret_p1,
                        'mv':mv, 'mv_adj':mv_adj}, index=[per])
    return res

In [31]:
# test some data
tic = time.perf_counter()
# g = comp[comp.gvkey==25056].groupby(['permno', 'fyear'])
code_list = comp.gvkey.unique().tolist()[:30]
test_comp = comp.set_index('gvkey').loc[code_list].reset_index()
g = test_comp.groupby(['permno', 'fyear'])

df_list = []
for name, group in tqdm_notebook(g):
    start = group.fysm.iloc[0].strftime('%Y-%m-%d')
    end = group.datadate.iloc[0].strftime('%Y-%m-%d')
    fystring = group.fystr.iloc[0]
    sample = crsp[crsp.permno==name[0]][start:end]
    if len(sample) > 0:
        r = sample.resample(fystring)
        df_list.append(r.apply(fiscal_resample, fys=fystring, per=name[0]))

df_test = pd.concat(df_list, axis=0)
df_test.index.names = ['date', 'permno']
toc= time.perf_counter()
print(toc - tic)


10.460685000000012


In [37]:
tic = time.perf_counter()
g = comp.groupby(['permno', 'fyear'])

df_list = []
    
for name, group in tqdm_notebook(g):

    start = group.fysm.iloc[0].strftime('%Y-%m-%d')
    end = group.datadate.iloc[0].strftime('%Y-%m-%d')
    fystring = group.fystr.iloc[0]
    sample = crsp[crsp.permno==name[0]][start:end]
    if len(sample) > 0:
        r = sample.resample(fystring)
        df_list.append(r.apply(fiscal_resample, fys=fystring, per=name[0]))

crsp1 = pd.concat(df_list, axis=0)
crsp1.index.names = ['date', 'permno']
# df_test
crsp1.to_csv('data/crsp_fa.csv')
toc= time.perf_counter()
print(toc - tic)


6685.1432568
